Word2Vec for Tamil words. 

This is to mainly to get the relation between the tamil words. 


In [0]:
import pandas as pd
import numpy as np 
import re
from matplotlib import pyplot as plt

In [0]:
import sys

In [0]:
sys.setrecursionlimit(2500)

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
%cd /gdrive/My\ Drive/Tamizh/data/wiki_data

/gdrive/My Drive/Tamizh/data/wiki_data


In [0]:
data = pd.read_csv("tawiki_data_df.csv")

In [0]:
#shape
data.shape, data.columns

((213758, 1), Index(['Content'], dtype='object'))

Convert each sentence to list of tokens. 

*observation : this have more then 2 crore vocabulary 
* Need to find the root words  - 

In [0]:
data['text_list'] = data.Content.str.split()

In [0]:
data.tail()

,Content,text_list
213753,கேப்டன் மார்வெல் ஆம் ஆண்டில் வெளியான அமெரிக்க ...,"[கேப்டன், மார்வெல், ஆம், ஆண்டில், வெளியான, அமெ..."
213754,வழிமாற்று திருகோணமலை பட்டினமும் சூழலும் பிரதேச...,"[வழிமாற்று, திருகோணமலை, பட்டினமும், சூழலும், ப..."
213755,ஆக்கெர்டைட்டு என்பது என்ற மூலக்கூற்று வாய்ப்பா...,"[ஆக்கெர்டைட்டு, என்பது, என்ற, மூலக்கூற்று, வாய..."
213756,NaN,NaN
213757,NaN,NaN


Remove NaN cols. .

In [0]:
data.dropna(inplace=True)

Get list of Unique words from the corpus

In [0]:
#Run this if you have good RAM

vocabulary = []

for i in range(data.shape[0]):
  #print(i)
  if(1%10000==0):
    print(i)
  vocabulary+=data.text_list.iloc[i]

  
  

In [0]:
len(vocabulary)

21151075

#Word2Vec

Generate word2vec representation for tokens in tamil. 



In [0]:
#Generate word 2 vec

from gensim.models import Word2Vec

In [0]:
texts = data.text_list.values

In [0]:
 
model = Word2Vec(texts, size=100, window=50, min_count=20, workers=6)

In [0]:
len(model.wv.vocab)

80860

In [0]:
model.wv.most_similar("ஐம்பது")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('முப்பது', 0.7912629842758179),
 ('நூறு', 0.7821017503738403),
 ('நாற்பது', 0.7711770534515381),
 ('பதினைந்து', 0.7171321511268616),
 ('ஆயிரத்து', 0.7063670754432678),
 ('எண்பது', 0.6890194416046143),
 ('பத்து', 0.6858401298522949),
 ('அறுபது', 0.6848776340484619),
 ('இரண்டரை', 0.6679751873016357),
 ('இலட்சம்', 0.6582279205322266)]

In [0]:
model.save("tawiki_w2v")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
